### 分词
1、正向最大匹配法：最大匹配是指以词典为依据，取词典中最长单词为第一个次取字数量的扫描串，在词典中进行扫描（为提升扫描效率，还可以跟据字数多少设计多个字典，然后根据字数分别从不同字典中进行扫描）。

2、逆向最大匹配法：即从后往前取词，类比正向最大匹配法。

3、双向最大匹配法：双向最大匹配法是将正向最大匹配法得到的分词结果和逆向最大匹配法的到的结果进行比较，从而决定正确的分词方法。


### n-gram

N-Gram是一种基于统计语言模型的算法。它的基本思想是将文本里面的内容按照字节进行大小为N的滑动窗口操作，形成了长度是N的字节片段序列。

每一个字节片段称为gram，对所有gram的出现频度进行统计，并且按照事先设定好的阈值进行过滤，形成关键gram列表，也就是这个文本的向量特征空间，列表中的每一种gram就是一个特征向量维度。

该模型基于这样一种假设，第N个词的出现只与前面N-1个词相关，而与其它任何词都不相关，整句的概率就是各个词出现概率的乘积。这些概率可以通过直接从语料中统计N个词同时出现的次数得到。

unigram:一元模型
bigram:二元模型
trigram:三元模型

参考链接：[自然语言处理中N-Gram模型介绍](https://zhuanlan.zhihu.com/p/32829048)

In [42]:
import pandas as pd
import numpy as np
import jieba
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer  

In [3]:
train_data = pd.read_csv("./cnews/cnews.train.txt",sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
test_data = pd.read_csv("./cnews/cnews.test.txt",sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
val_data = pd.read_csv("./cnews/cnews.val.txt",sep='\t',engine='python',names=['label','content'],encoding='UTF-8')

In [4]:
train_data.head(5)

,label,content
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,体育,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,体育,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,体育,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,体育,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...


In [19]:
# 分词
seg_list = jieba.cut(train_data["content"][0], cut_all=False)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

Full Mode: 马/ 晓/ 旭/ 意外/ 受伤/ 让/ 国奥/ 警惕/ / / 无奈/ 大雨/ 格外/ 青睐/ 殷家/ 军/ 记者/ 傅/ 亚/ 雨/ 沈阳/ 报道/ / / 来到/ 沈阳/ / / 国奥/ 国奥队/ 依然/ 没有/ 摆脱/ 雨水/ 的/ 困扰/ / 7/ 月/ 31/ 日/ 下午/ 6/ 点/ / / 国奥/ 国奥队/ 的/ 日常/ 训练/ 再度/ 受到/ 大雨/ 的/ 干扰/ / / 无奈/ 之下/ 队员/ 们/ 只/ 慢跑/ 了/ 25/ 分钟/ 就/ 草草/ 草草收场/ 收场/ / 31/ 日/ 上午/ 10/ 点/ / / 国奥/ 国奥队/ 在/ 奥体/ 奥体中心/ 中心/ 外场/ 训练/ 的/ 时候/ / / 天/ 就是/ 阴沉/ 阴沉沉/ 沉沉/ 沉沉的/ / / 气象/ 气象预报/ 预报/ 显示/ 当天/ 天下/ 下午/ 沈阳/ 就/ 有/ 大雨/ / / 但/ 幸好/ 队伍/ 上午/ 的/ 训练/ 并/ 没有/ 受到/ 到任/ 任何/ 何干/ 干扰/ / / 下午/ 6/ 点/ / / 当/ 球队/ 抵达/ 训练/ 训练场/ 时/ / / 大雨/ 已经/ 下/ 了/ 几个/ 小时/ / / 而且/ 丝毫/ 没有/ 停下/ 停下来/ 下来/ 的/ 意思/ / / 抱/ 着/ 试一试/ 的/ 态度/ / / 球队/ 开始/ 了/ 当天/ 天下/ 下午/ 的/ 例行/ 训练/ / 25/ 分钟/ 过去/ 了/ / / 天气/ 没有/ 任何/ 转/ 好/ 的/ 迹象/ / / 为了/ 保护/ 球员/ 们/ / / 国奥/ 国奥队/ 决定/ 中止/ 当天/ 的/ 训练/ / / 全队/ 立即/ 返回/ 酒店/ / / 在/ 雨/ 中/ 训练/ 对/ 足球/ 足球队/ 球队/ 来说/ 并/ 不是/ 什么/ 稀罕/ 罕事/ / / 但/ 在/ 奥运/ 奥运会/ 即将/ 开始/ 之前/ / / 全队/ 变得/ / / 娇贵/ / / 了/ / / 在/ 沈阳/ 最后/ 一周/ 的/ 训练/ / / 国奥/ 国奥队/ 首先/ 先要/ 保证/ 现有/ 的/ 球员/ 不再/ 出现/ 出现意外/ 意外/ 的/ 伤病/ 病情/ 情况/ 以免/ 影响/ 正式/ 比赛/ / / 因此/ 这/ 一/ 阶段/ 控制/ 训练/ 受伤/ / / 控制/ 感冒/ 等

In [38]:
# 读取停用词
stopwords = []
with open('stopwords.txt', 'r') as fr:
    for line in fr:
        stopwords.append(line[:-1])

In [46]:
# 计算词频
seg_list = jieba.cut(train_data["content"][0], cut_all=False)
result = []
for seg in seg_list:
    seg = ''.join(seg.split())
    if seg != '' and seg != "\n" and seg != "\n\n" and seg not in stopwords:
        result.append(seg)
word_fre = {}
for i in set(result):
    word_fre[i] = result.count(i)
word_fre = sorted(word_fre.items(), key=lambda item: item[1], reverse=True)
word_fre

[('训练', 13),
 ('国奥队', 8),
 ('沈阳', 8),
 ('大雨', 5),
 ('下午', 4),
 ('中', 4),
 ('感冒', 4),
 ('球员', 4),
 ('日', 4),
 ('冯萧霆', 4),
 ('雨水', 3),
 ('受伤', 3),
 ('当天', 3),
 ('队员', 3),
 ('点', 3),
 ('长春', 3),
 ('队伍', 3),
 ('抵达', 2),
 ('月', 2),
 ('球队', 2),
 ('国奥', 2),
 ('热身赛', 2),
 ('无奈', 2),
 ('一位', 2),
 ('31', 2),
 ('25', 2),
 ('6', 2),
 ('分钟', 2),
 ('青睐', 2),
 ('控制', 2),
 ('上午', 2),
 ('干扰', 2),
 ('警惕', 2),
 ('雨', 2),
 ('7', 2),
 ('事情', 2),
 ('马晓旭', 2),
 ('全队', 2),
 ('例子', 1),
 ('再度', 1),
 ('保护', 1),
 ('疾病', 1),
 ('非战斗', 1),
 ('介绍', 1),
 ('例行', 1),
 ('报道', 1),
 ('现有', 1),
 ('日常', 1),
 ('之下', 1),
 ('29', 1),
 ('几个', 1),
 ('嘱咐', 1),
 ('参加', 1),
 ('担心', 1),
 ('再出', 1),
 ('依然', 1),
 ('正式', 1),
 ('训练场', 1),
 ('恢复', 1),
 ('阴沉沉', 1),
 ('显示', 1),
 ('走', 1),
 ('10', 1),
 ('转好', 1),
 ('困扰', 1),
 ('出现意外', 1),
 ('抱', 1),
 ('迹象', 1),
 ('停下来', 1),
 ('前科', 1),
 ('态度', 1),
 ('幸好', 1),
 ('军', 1),
 ('这一', 1),
 ('来到', 1),
 ('草草收场', 1),
 ('女足', 1),
 ('动作', 1),
 ('事', 1),
 ('意外', 1),
 ('天', 1),
 ('变得', 1),
 ('搅和', 1),
 ('

In [85]:
# 分词
train_data = train_data[:5]
words = []
for sentence in train_data['content']:
    word = list(jieba.cut(sentence))
    for w in list(set(word) and set(stopwords)):
        while w in word:
            word.remove(w)
    words.append(' '.join(word))
train_data['content_'] = words

In [86]:
# 计算 TF-IDF
count_vectorizer = CountVectorizer(max_features=256, min_df=2)
count_vectorizer.fit_transform(train_data['content_'])
fea_vec = count_vectorizer.transform(train_data['content_']).toarray()
fea_vec

array([[ 1,  0,  0,  0,  0,  2,  1,  2,  4,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1,  0,  1,  3,  0,  1,  0,  1,  1,  1,  1,
         0,  3,  1,  0,  1,  1,  0,  0,  2,  0,  0,  0,  0,  0,  2,  1,
         0,  1,  0,  1,  1,  8,  0,  2,  0,  4,  2,  0,  0,  0, 13,  1,
         0,  0,  0,  1,  0,  0,  3,  1],
       [ 0,  0,  1,  1,  1,  0,  0,  0,  0,  1,  1,  3,  5,  1,  1,  1,
         0,  3,  0,  2,  0,  1,  0,  0,  1,  1,  1,  1,  3,  3,  1,  1,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  2,  0,  0,
         1,  2,  0,  0,  4,  0,  0,  2,  1,  2,  0,  0,  1,  0,  1,  3,
         1,  0,  0,  0,  1,  1,  1,  1],
       [ 1,  1,  0,  0,  0,  0,  1,  0,  0,  1,  1,  0,  1,  0,  1,  1,
         1,  0,  4,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  0,  1,  0,  0,  0,  2,  1,  5,  1,  3,  1,  0,  3,  1,
         1,  0,  2,  2,  0,  0,  0,  0,  3,  0,  3,  1,  0,  1,  2,  0,
         0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  